In [1]:
import tensorflow as tf
import string
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import os

2023-05-15 23:26:59.744764: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-15 23:26:59.928315: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-15 23:26:59.929673: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-15 23:27:01.059074: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data = ''
with open('./asoif.txt') as f:
    lines = ''.join(f.readlines())
    lines = lines.replace('\n', 'å')
    lines = re.sub('\å', ' å ', lines)
    lines = re.sub('\s+', ' ', lines)
    
    data = lines

In [3]:
data = data[:16566]

In [4]:
def clean(doc):
    tokens = doc.split()
    
    #removes all puncutation
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [w.lower() for w in tokens if w.isalpha()]
    
    return tokens

In [5]:
tokens = clean(data)

In [6]:
length = 50 + 1
lines = []

for i in range(length, len(tokens)):
               seq = tokens[i - length: i]
               line = ' '.join(seq)
               
               lines.append(line)
               if i > 10000:
                   break

In [7]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
sequences = np.array(sequences)

In [8]:
X = sequences[:, :-1]
y = sequences[:, -1]

In [10]:
vocab_size = len(tokenizer.word_index) + 1
y = to_categorical(y, num_classes=vocab_size)

In [11]:
model = None
if os.path.isfile('./model.h5'):
    model = load_model('./model.h5')
else:
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=50, input_length=X.shape[1]))

    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(100, return_sequences=False))

    model.add(Dense(250, activation='relu'))
    model.add(Dense(125, activation='relu'))
    model.add(Dense(vocab_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2023-05-15 23:27:03.461975: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-15 23:27:03.740657: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-15 23:27:03.742372: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-15 23:27:03.743643: I tensorflow/core/com

In [12]:
model.fit(X, y, epochs=50, batch_size=256, verbose=1)

Epoch 1/50


2023-05-15 23:27:04.464938: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-15 23:27:04.466584: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-15 23:27:04.468277: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

12/12 [==============================] - 4s 141ms/step - loss: 0.5117 - accuracy: 0.8511
Epoch 2/50
12/12 [==============================] - 2s 136ms/step - loss: 0.5348 - accuracy: 0.8373
Epoch 3/50
12/12 [==============================] - 2s 136ms/step - loss: 0.5672 - accuracy: 0.8274
Epoch 4/50
12/12 [==============================] - 2s 136ms/step - loss: 0.5503 - accuracy: 0.8308
Epoch 5/50
12/12 [==============================] - 2s 137ms/step - loss: 0.5619 - accuracy: 0.8315
Epoch 6/50
12/12 [==============================] - 2s 138ms/step - loss: 0.5733 - accuracy: 0.8287
Epoch 7/50
12/12 [==============================] - 2s 140ms/step - loss: 0.6012 - accuracy: 0.8133
Epoch 8/50
12/12 [==============================] - 2s 140ms/step - loss: 0.5793 - accuracy: 0.8232
Epoch 9/50
12/12 [==============================] - 2s 141ms/step - loss: 0.5919 - accuracy: 0.8150
Epoch 10/50
12/12 [==============================] - 2s 140ms/step - loss: 0.5952 - accuracy: 0.8157
Epoch 11/5

KeyboardInterrupt: 

In [ ]:
model.save('./model.h5')